In [11]:
import pandas as pd
from pathlib import Path
import numpy as np
import json

# ============
# CONFIG
# ============
# Aponte a pasta onde estão seus CSVs do MIMIC (extraídos do PhysioNet)
DATA_DIR = Path("../rascunho/mimic-iii-clinical-database-demo-1.4/")

USE_MIMIC_IV = False  # se False, usa nomes de arquivos do MIMIC-III

# Nomes de arquivo por versão
if USE_MIMIC_IV:
    f_patients     = DATA_DIR / "core" / "patients.csv"
    f_admissions   = DATA_DIR / "core" / "admissions.csv"
    f_diagnoses    = DATA_DIR / "hosp" / "diagnoses_icd.csv"
    f_d_icd        = DATA_DIR / "hosp" / "d_icd_diagnoses.csv"
    f_labevents    = DATA_DIR / "hosp" / "labevents.csv"
    f_d_labitems   = DATA_DIR / "hosp" / "d_labitems.csv"
    f_noteevents   = DATA_DIR / "note" / "noteevents.csv"
    f_prescriptions= DATA_DIR / "hosp" / "prescriptions.csv"
    ADMIT_COL, DISCH_COL = "admittime", "dischtime"
    CHARTTIME_COL = "charttime"
    START_COL, STOP_COL = "starttime", "stoptime"
    HAS_ANCHOR = True
else:
    # MIMIC-III
    f_patients     = DATA_DIR / "PATIENTS.csv"
    f_admissions   = DATA_DIR / "ADMISSIONS.csv"
    f_diagnoses    = DATA_DIR / "DIAGNOSES_ICD.csv"
    f_d_icd        = DATA_DIR / "D_ICD_DIAGNOSES.csv"
    f_labevents    = DATA_DIR / "LABEVENTS.csv"
    f_d_labitems   = DATA_DIR / "D_LABITEMS.csv"
    f_noteevents   = DATA_DIR / "NOTEEVENTS.csv"
    f_prescriptions= DATA_DIR / "PRESCRIPTIONS.csv"
    ADMIT_COL, DISCH_COL = "ADMITTIME", "DISCHTIME"
    CHARTTIME_COL = "CHARTTIME"
    START_COL, STOP_COL = "STARTDATE", "ENDDATE"
    HAS_ANCHOR = False

print("Carregando dados...")

# ============
# LOAD (colunas essenciais para reduzir memória)
# ============
patients = pd.read_csv(f_patients, usecols=lambda c: c.upper() in {
    "SUBJECT_ID","GENDER","ANCHOR_AGE","ANCHOR_YEAR","ANCHOR_YEAR_GROUP","DOB"
}, low_memory=False)

admissions = pd.read_csv(f_admissions,
                         usecols=lambda c: c.upper() in {
                             "SUBJECT_ID","HADM_ID", ADMIT_COL.upper(), DISCH_COL.upper(),
                             "ADMISSION_TYPE","INSURANCE","LANGUAGE",
                             "MARITAL_STATUS","ETHNICITY"
                         }, low_memory=False)

diagnoses = pd.read_csv(f_diagnoses, usecols=lambda c: c.upper() in {
    "SUBJECT_ID","HADM_ID","ICD_CODE","ICD9_CODE","SEQ_NUM"
}, low_memory=False)

d_icd = pd.read_csv(f_d_icd, usecols=lambda c: c.upper() in {
    "ICD_CODE","ICD9_CODE","LONG_TITLE"
}, low_memory=False)

labevents = pd.read_csv(f_labevents,
                        usecols=lambda c: c.upper() in {
                            "SUBJECT_ID","HADM_ID","ITEMID","VALUENUM","VALUEUOM",
                            CHARTTIME_COL.upper(),"CHARTDATE"
                        }, low_memory=False)

d_labitems = pd.read_csv(f_d_labitems, usecols=lambda c: c.upper() in {"ITEMID","LABEL"}, low_memory=False)

noteevents = pd.read_csv(f_noteevents,
                         usecols=lambda c: c.upper() in {
                             "SUBJECT_ID","HADM_ID",CHARTTIME_COL.upper(),"CATEGORY","ISERROR","TEXT"
                         }, low_memory=False)

prescriptions = pd.read_csv(f_prescriptions, low_memory=False)

# Normaliza nomes (minúsculos)
def normcols(df):
    df.columns = [c.lower() for c in df.columns]
    return df

patients, admissions = normcols(patients), normcols(admissions)
diagnoses, d_icd = normcols(diagnoses), normcols(d_icd)
labevents, d_labitems = normcols(labevents), normcols(d_labitems)
noteevents, prescriptions = normcols(noteevents), normcols(prescriptions)

# Converte datas APÓS normalizar os nomes
admit_col, disch_col = ADMIT_COL.lower(), DISCH_COL.lower()
charttime_col = CHARTTIME_COL.lower()
start_col, stop_col = START_COL.lower(), STOP_COL.lower()

if admit_col in admissions.columns:
    admissions[admit_col] = pd.to_datetime(admissions[admit_col], errors='coerce')
if disch_col in admissions.columns:
    admissions[disch_col] = pd.to_datetime(admissions[disch_col], errors='coerce')
if charttime_col in labevents.columns:
    labevents[charttime_col] = pd.to_datetime(labevents[charttime_col], errors='coerce')
if charttime_col in noteevents.columns:
    noteevents[charttime_col] = pd.to_datetime(noteevents[charttime_col], errors='coerce')
if start_col in prescriptions.columns:
    prescriptions[start_col] = pd.to_datetime(prescriptions[start_col], errors='coerce')
if stop_col in prescriptions.columns:
    prescriptions[stop_col] = pd.to_datetime(prescriptions[stop_col], errors='coerce')

print("Processando idade estimada...")
# ============
# Idade estimada
# ============
if HAS_ANCHOR and "anchor_age" in patients.columns:
    patients["age_estimada"] = patients["anchor_age"].round().astype("Int64")
else:
    if "dob" in patients.columns:
        patients["dob"] = pd.to_datetime(patients["dob"], errors="coerce")
        first_adm = admissions.sort_values(admit_col).groupby("subject_id", as_index=False)[admit_col].first()
        patients = patients.merge(first_adm, on="subject_id", how="left")
        
        try:
            age_days = (patients[admit_col] - patients["dob"]).dt.days
            age_days = age_days.clip(lower=0, upper=120*365.25)
            patients["age_estimada"] = (age_days / 365.25).round().astype("Int64")
        except:
            patients["age_estimada"] = pd.NA
        
        patients.drop(columns=[admit_col], inplace=True, errors='ignore')
    else:
        patients["age_estimada"] = pd.NA

print("Processando diagnósticos...")
# ============
# Diagnósticos
# ============
icd_key_left  = "icd9_code"  if "icd9_code"  in diagnoses.columns else "icd_code"
icd_key_right = "icd9_code"  if "icd9_code"  in d_icd.columns     else "icd_code"

diag = diagnoses.merge(d_icd, left_on=icd_key_left, right_on=icd_key_right, how="left")
diag["icd_str"] = diag[icd_key_left].astype(str) + " (" + diag["long_title"].fillna("sem_titulo") + ")"

diag_sorted = diag.sort_values(["hadm_id","seq_num"])
diag_grouped = diag_sorted.groupby("hadm_id")["icd_str"].apply(lambda x: "; ".join(x.astype(str)))
diag_list = pd.DataFrame({"hadm_id": diag_grouped.index, "diag_list": diag_grouped.values})

print("Processando labs...")
# ============
# Labs essenciais
# ============
LAB_KEYS = ["creatin", "potass", "sod", "hemogl"]
lab_join = labevents.merge(d_labitems, on="itemid", how="left")
time_in = admissions[["hadm_id", admit_col]]
lab_join = lab_join.merge(time_in, on="hadm_id", how="left")
lab_window = lab_join[
    (lab_join["valuenum"].notna()) &
    (lab_join["hadm_id"].notna()) &
    (lab_join[charttime_col].notna()) &
    (lab_join[charttime_col].between(lab_join[admit_col], lab_join[admit_col] + pd.Timedelta(hours=48)))
].copy()

lab_window["label"] = lab_window["label"].str.lower()
lab_window = lab_window[lab_window["label"].apply(lambda s: any(k in str(s) for k in LAB_KEYS))]

lab_last = lab_window.sort_values(charttime_col).groupby(["hadm_id","label"]).tail(1)

def lab_alias(lbl: str) -> str:
    if pd.isna(lbl):
        return "unknown"
    lbl = str(lbl).lower()
    if "creatin" in lbl:  return "creatinina"
    if "potass"  in lbl:  return "potassio"
    if "sod"     in lbl:  return "sodio"
    if "hemogl"  in lbl:  return "hemoglobina"
    return lbl

lab_last["alias"] = lab_last["label"].apply(lab_alias)
labs_val = lab_last.pivot_table(index="hadm_id", columns="alias", values="valuenum", aggfunc="last")
labs_uom = lab_last.pivot_table(index="hadm_id", columns="alias", values="valueuom", aggfunc=lambda x: x.dropna().iloc[-1] if len(x.dropna()) else None)

labs_val.columns = [c for c in labs_val.columns]
labs_uom.columns = [f"{c}_uom" for c in labs_uom.columns]
labs = labs_val.join(labs_uom, how="outer").reset_index()

print("Processando notas clínicas...")
# ============
# Notas clínicas
# ============
notes = noteevents.copy()
notes = notes[notes["iserror"].fillna(0) != 1]
notes = notes.merge(time_in, on="hadm_id", how="left")
notes_24h = notes[
    (notes[charttime_col].notna()) &
    (notes[charttime_col].between(notes[admit_col], notes[admit_col] + pd.Timedelta(hours=24)))
].copy()

def resumir_bloco(df_hadm, limit_chars=800):
    parts = []
    for _, r in df_hadm.sort_values(charttime_col, ascending=False).iterrows():
        txt = str(r.get("text", ""))
        txt = " ".join(txt.split())
        parts.append(f"[{r[charttime_col]:%Y-%m-%d %H:%M} {r.get('category','')}]\n{txt[:limit_chars]}")
    return "\n---\n".join(parts) if parts else None

if len(notes_24h) > 0:
    notas_list = []
    for hadm_id, group in notes_24h.groupby("hadm_id"):
        resumo = resumir_bloco(group)
        notas_list.append({"hadm_id": hadm_id, "notas_24h_resumo": resumo})
    notas_resumo = pd.DataFrame(notas_list)
else:
    notas_resumo = pd.DataFrame(columns=["hadm_id", "notas_24h_resumo"])

print("Consolidando prontuário...")
# ============
# PRONTUÁRIO CONSOLIDADO
# ============
prontuario = (admissions
              .merge(patients[["subject_id","gender","age_estimada"]], on="subject_id", how="left")
              .merge(diag_list, on="hadm_id", how="left")
              .merge(labs, on="hadm_id", how="left")
              .merge(notas_resumo, on="hadm_id", how="left"))

print("Processando prescrições...")
# ============
# PRESCRIÇÕES ASSOCIADAS
# ============
RX_KEEP = [c for c in [
    "subject_id","hadm_id","drug","drug_name_poe","drug_type",
    "dose_val_rx","dose_unit_rx","form_rx","route",
    start_col, stop_col
] if c in prescriptions.columns]

rx = prescriptions[RX_KEEP].copy()

if "drug" in rx.columns:
    rx["drug_norm"] = rx["drug"].astype(str)
if "drug_name_poe" in rx.columns:
    rx["drug_norm"] = rx["drug_name_poe"].fillna(rx["drug_norm"])

def to_rx_obj(row):
    return {
        "drug":       row.get("drug_norm"),
        "drug_type":  row.get("drug_type"),
        "dose_val":   None if "dose_val_rx" not in row else row.get("dose_val_rx"),
        "dose_unit":  None if "dose_unit_rx" not in row else row.get("dose_unit_rx"),
        "form":       row.get("form_rx"),
        "route":      row.get("route"),
        "starttime":  str(row.get(start_col)) if pd.notna(row.get(start_col)) else None,
        "stoptime":   str(row.get(stop_col))  if pd.notna(row.get(stop_col))  else None,
    }

if len(rx) > 0:
    rx_list = []
    for hadm_id, group in rx.groupby("hadm_id"):
        prescricoes = [to_rx_obj(r) for _, r in group.iterrows()]
        rx_list.append({"hadm_id": hadm_id, "prescricoes": prescricoes})
    rx_grouped = pd.DataFrame(rx_list)
else:
    rx_grouped = pd.DataFrame(columns=["hadm_id", "prescricoes"])

payload = prontuario.merge(rx_grouped, on="hadm_id", how="left")
payload["prescricoes"] = payload["prescricoes"].apply(lambda x: x if isinstance(x, list) else [])

# ============
# RESUMO
# ============
print("\n" + "=" * 60)
print("RESUMO DOS DATAFRAMES GERADOS")
print("=" * 60)
print(f"\n1. PRONTUÁRIO:")
print(f"   - Total de prontuários: {len(prontuario)}")
print(f"   - Colunas: {list(prontuario.columns)}")

print(f"\n2. PRESCRIÇÕES (granular):")
print(f"   - Total de prescrições: {len(rx)}")
print(f"   - Colunas: {list(rx.columns)}")

print(f"\n3. PAYLOAD (consolidado):")
print(f"   - Total de payloads: {len(payload)}")
print(f"   - Colunas: {list(payload.columns)}")

# Exemplo
if len(payload) > 0:
    ex = payload.sample(1, random_state=0).to_dict(orient="records")[0]
    print("\n" + "=" * 60)
    print("EXEMPLO DE PAYLOAD (1 prontuário com prescrições)")
    print("=" * 60)
    print(json.dumps(ex, ensure_ascii=False, indent=2, default=str))


Carregando dados...
Processando idade estimada...
Processando diagnósticos...
Processando labs...
Processando notas clínicas...
Consolidando prontuário...
Processando prescrições...
Processando notas clínicas...
Consolidando prontuário...
Processando prescrições...

RESUMO DOS DATAFRAMES GERADOS

1. PRONTUÁRIO:
   - Total de prontuários: 129
   - Colunas: ['subject_id', 'hadm_id', 'admittime', 'dischtime', 'admission_type', 'insurance', 'language', 'marital_status', 'ethnicity', 'gender', 'age_estimada', 'diag_list', 'creatinina', 'hemoglobina', 'potassio', 'sodio', 'creatinina_uom', 'hemoglobina_uom', 'potassio_uom', 'sodio_uom', 'notas_24h_resumo']

2. PRESCRIÇÕES (granular):
   - Total de prescrições: 10398
   - Colunas: ['subject_id', 'hadm_id', 'drug', 'drug_name_poe', 'drug_type', 'dose_val_rx', 'dose_unit_rx', 'route', 'startdate', 'enddate', 'drug_norm']

3. PAYLOAD (consolidado):
   - Total de payloads: 129
   - Colunas: ['subject_id', 'hadm_id', 'admittime', 'dischtime', 'adm

In [26]:
rx

,subject_id,hadm_id,drug,drug_name_poe,drug_type,dose_val_rx,dose_unit_rx,route,startdate,enddate,drug_norm
0,42458,159647,Pneumococcal Vac Polyvalent,Pneumococcal Vac Polyvalent,MAIN,0.5,mL,IM,2146-07-21,2146-07-22,Pneumococcal Vac Polyvalent
1,42458,159647,Bisacodyl,Bisacodyl,MAIN,10,mg,PO,2146-07-21,2146-07-22,Bisacodyl
2,42458,159647,Bisacodyl,Bisacodyl,MAIN,10,mg,PR,2146-07-21,2146-07-22,Bisacodyl
3,42458,159647,Senna,Senna,MAIN,1,TAB,PO,2146-07-21,2146-07-22,Senna
4,42458,159647,Docusate Sodium (Liquid),Docusate Sodium (Liquid),MAIN,100,mg,PO,2146-07-21,2146-07-21,Docusate Sodium (Liquid)
...,...,...,...,...,...,...,...,...,...,...,...
10393,42430,100969,Sodium Chloride 0.9% Flush,Sodium Chloride 0.9% Flush,MAIN,3,mL,IV,2142-11-29,2142-11-30,Sodium Chloride 0.9% Flush
10394,42430,100969,Acetaminophen,Acetaminophen,MAIN,650,mg,PR,2142-11-30,2142-11-30,Acetaminophen
10395,42430,100969,0.9% Sodium Chloride,NaN,BASE,1000,mL,IV,2142-11-26,2142-11-27,0.9% Sodium Chloride
10396,42430,100969,D5W,NaN,BASE,250,mL,IV,2142-11-26,2142-11-27,D5W


In [19]:
def format_for_llm(row):
    prompt = f"""
    PACIENTE ID: {row['subject_id']} | Admissão: {row['hadm_id']}
    
    DADOS DEMOGRÁFICOS:
    - Idade: {row['age_estimada']} anos
    - Gênero: {row['gender']}
    - Etnia: {row['ethnicity']}
    - Estado civil: {row['marital_status']}
    
    ADMISSÃO:
    - Tipo: {row['admission_type']}
    - Data admissão: {row['admittime']}
    - Data alta: {row['dischtime']}
    - Seguro: {row['insurance']}
    
    DIAGNÓSTICOS:
    {row['diag_list']}
    
    EXAMES LABORATORIAIS (primeiras 48h):
    - Creatinina: {row['creatinina']} {row['creatinina_uom']}
    - Hemoglobina: {row['hemoglobina']} {row['hemoglobina_uom']}
    - Potássio: {row['potassio']} {row['potassio_uom']}
    - Sódio: {row['sodio']} {row['sodio_uom']}
    
    NOTAS CLÍNICAS (primeiras 24h):
    {row['notas_24h_resumo']}
    """
    return prompt

In [24]:
prontuario

,subject_id,hadm_id,admittime,dischtime,admission_type,insurance,language,marital_status,ethnicity,gender,...,diag_list,creatinina,hemoglobina,potassio,sodio,creatinina_uom,hemoglobina_uom,potassio_uom,sodio_uom,notas_24h_resumo
0,10006,142345,2164-10-23 21:09:00,2164-11-01 17:15:00,EMERGENCY,Medicare,NaN,SEPARATED,BLACK/AFRICAN AMERICAN,F,...,99591 (Sepsis); 99662 (Infection and inflammat...,5.3,11.1,4.6,138.0,mg/dL,g/dL,mEq/L,mEq/L,NaN
1,10011,105331,2126-08-14 22:32:00,2126-08-28 18:59:00,EMERGENCY,Private,NaN,SINGLE,UNKNOWN/NOT SPECIFIED,F,...,570 (Acute and subacute necrosis of liver); 07...,0.7,9.8,4.0,138.0,mg/dL,g/dL,mEq/L,mEq/L,NaN
2,10013,165520,2125-10-04 23:36:00,2125-10-07 15:13:00,EMERGENCY,Medicare,NaN,NaN,UNKNOWN/NOT SPECIFIED,F,...,0389 (Unspecified septicemia); 41071 (Subendoc...,276.0,8.8,4.1,138.0,IU/L,g/dL,mEq/L,mEq/L,NaN
3,10017,199207,2149-05-26 17:19:00,2149-06-03 18:42:00,EMERGENCY,Medicare,NaN,DIVORCED,WHITE,F,...,81201 (Closed fracture of surgical neck of hum...,0.4,8.8,4.0,140.0,mg/dL,g/dL,mEq/L,mEq/L,NaN
4,10019,177759,2163-05-14 20:43:00,2163-05-15 12:00:00,EMERGENCY,Medicare,NaN,DIVORCED,WHITE,M,...,0389 (Unspecified septicemia); 51881 (Acute re...,4.0,12.5,5.2,136.0,mg/dL,g/dL,mEq/L,mEq/L,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124,44083,198330,2112-05-28 15:45:00,2112-06-07 16:50:00,EMERGENCY,Private,ENGL,SINGLE,WHITE,M,...,4239 (Unspecified disease of pericardium); 511...,0.6,8.8,4.1,138.0,mg/dL,g/dL,mEq/L,mEq/L,NaN
125,44154,174245,2178-05-14 20:29:00,2178-05-15 09:45:00,EMERGENCY,Medicare,ENGL,MARRIED,WHITE,M,...,0389 (Unspecified septicemia); 5185 (sem_titul...,0.9,14.0,4.5,142.0,mg/dL,g/dL,mEq/L,mEq/L,NaN
126,44212,163189,2123-11-24 14:14:00,2123-12-30 14:31:00,EMERGENCY,Medicare,ENGL,SINGLE,BLACK/AFRICAN AMERICAN,F,...,03811 (Methicillin susceptible Staphylococcus ...,1.9,7.9,3.6,135.0,mg/dL,g/dL,mEq/L,mEq/L,NaN
127,44222,192189,2180-07-19 06:55:00,2180-07-20 13:00:00,EMERGENCY,Medicare,ENGL,SINGLE,WHITE,M,...,42781 (Sinoatrial node dysfunction); 5849 (Acu...,1.2,12.9,4.2,135.0,mg/dL,g/dL,mEq/L,mEq/L,NaN


In [ ]:
# # Visualizar os dados gerados
# print("📊 VISÃO GERAL DOS DATAFRAMES")
# print("\n1️⃣ PRONTUÁRIO (demografia + diagnósticos + labs + notas):")
# print(prontuario.head(3))
# print(f"\nShape: {prontuario.shape}")

# print("\n" + "="*80)
# print("2️⃣ PRESCRIÇÕES (formato granular):")
# print(rx.head(3))
# print(f"\nShape: {rx.shape}")

# print("\n" + "="*80)
# print("3️⃣ PAYLOAD (prontuário + prescrições consolidadas):")
# print(payload[['subject_id', 'hadm_id', 'gender', 'age_estimada']].head(3))
# print(f"\nShape: {payload.shape}")

# # Exemplo de um paciente específico
# print("\n" + "="*80)
# print("📋 EXEMPLO DETALHADO DE UM PACIENTE:")
# exemplo = payload.iloc[0]
# print(f"\nSubject ID: {exemplo['subject_id']}")
# print(f"HADM ID: {exemplo['hadm_id']}")
# print(f"Gênero: {exemplo['gender']}")
# print(f"Idade estimada: {exemplo['age_estimada']}")
# print(f"Tipo de admissão: {exemplo.get('admission_type', 'N/A')}")
# print(f"\nDiagnósticos: {exemplo.get('diag_list', 'N/A')[:200]}...")
# print(f"\nNúmero de prescrições: {len(exemplo['prescricoes'])}")
# if len(exemplo['prescricoes']) > 0:
#     print(f"Primeira prescrição: {exemplo['prescricoes'][0]}")


In [16]:
# # Salvar os dataframes (opcional)
# # Descomente as linhas abaixo para salvar os dados

# # prontuario.to_csv("prontuario_consolidado.csv", index=False)
# # rx.to_csv("prescricoes_granular.csv", index=False)
# # payload.to_json("payload_completo.json", orient="records", indent=2, default_handler=str)

# print("✅ Dataframes prontos para uso!")
# print("\nDataframes disponíveis:")
# print("  - prontuario: dados demográficos + diagnósticos + labs + notas")
# print("  - rx: prescrições em formato granular (uma linha por prescrição)")
# print("  - payload: prontuário consolidado com prescrições em lista (ideal para LLMs)")


In [37]:
# Save payload in ../data/payload_mimic3.json as records
payload.to_json("../data/payload_mimic3.json", orient="records", indent=2, default_handler=str)

In [4]:
import json 
import pandas as pd

# load payload
with open("../data/payload_mimic3.json", "r") as f:
    loaded_payload = json.load(f)

In [5]:
loaded_df = pd.DataFrame(loaded_payload)